In [5]:
# Import Libraries
import pygame
import random
import sys


In [ ]:
# Initialize Pygame
pygame.init()

# Screen Dimensions
SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 1000

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
PINK = (255, 0, 255)
CONFETTI_COLORS = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255)]

# Screen Setup
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("myimage.png")

# Block Settings
BLOCK_SIZE = 30
ROWS = SCREEN_HEIGHT // BLOCK_SIZE
COLS = SCREEN_WIDTH // BLOCK_SIZE

# Player Settings
player_img = pygame.image.load("myimage.png")  # Load your player image here
player_img = pygame.transform.scale(player_img, (BLOCK_SIZE, BLOCK_SIZE))  # Scale to block size
player_x, player_y = 0, 0

# Exit Settings
exit_img = pygame.image.load("shiwu.png")  # Load your exit image here
exit_img = pygame.transform.scale(exit_img, (BLOCK_SIZE, BLOCK_SIZE))  # Scale to block size
exit_x, exit_y = COLS - 1, ROWS - 1

# Timer
start_time = pygame.time.get_ticks()
time_limit = 60  # seconds

# Font
font = pygame.font.Font(None, 36)

# Maze Generation
def generate_maze(rows, cols):
    maze = [[1 for _ in range(cols)] for _ in range(rows)]

    def carve_path(x, y):
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        random.shuffle(directions)
        for dx, dy in directions:
            nx, ny = x + dx * 2, y + dy * 2
            if 0 <= nx < rows and 0 <= ny < cols and maze[nx][ny] == 1:
                maze[x + dx][y + dy] = 0
                maze[nx][ny] = 0
                carve_path(nx, ny)

    maze[0][0] = 0  # Start point
    carve_path(0, 0)
    maze[rows - 1][cols - 1] = 0  # Exit point
    return maze

maze = generate_maze(ROWS, COLS)

# Draw Maze
def draw_maze(maze):
    for i in range(ROWS):
        for j in range(COLS):
            color = WHITE if maze[i][j] == 0 else BLACK
            pygame.draw.rect(screen, color, (j * BLOCK_SIZE, i * BLOCK_SIZE, BLOCK_SIZE, BLOCK_SIZE))

# Celebration Animation
def celebration():
    confetti = []
    for _ in range(100):  # Generate random confetti particles
        x = random.randint(0, SCREEN_WIDTH)
        y = random.randint(0, SCREEN_HEIGHT)
        color = random.choice(CONFETTI_COLORS)
        confetti.append((x, y, color))
    
    for _ in range(300):  # Adjusted for ~10 seconds (300 frames at 30ms delay each)
        screen.fill(BLACK)
        for x, y, color in confetti:
            pygame.draw.circle(screen, color, (x, y), 5)
        confetti = [(x, y + 5, color) for x, y, color in confetti]  # Move confetti downward
        love_text = font.render("Finally, you got your love!", True, PINK)
        screen.blit(love_text, (SCREEN_WIDTH // 2 - 150, SCREEN_HEIGHT // 2 - 20))
        pygame.display.flip()
        pygame.time.delay(30)


# Game Loop
running = True
while running:
    # Event Handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Key Handling
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT] and player_x > 0 and maze[player_y][player_x - 1] == 0:
        player_x -= 1
    if keys[pygame.K_RIGHT] and player_x < COLS - 1 and maze[player_y][player_x + 1] == 0:
        player_x += 1
    if keys[pygame.K_UP] and player_y > 0 and maze[player_y - 1][player_x] == 0:
        player_y -= 1
    if keys[pygame.K_DOWN] and player_y < ROWS - 1 and maze[player_y + 1][player_x] == 0:
        player_y += 1

    # Check Victory
    if player_x == exit_x and player_y == exit_y:
        celebration()
        running = False

    # Timer Countdown
    elapsed_time = (pygame.time.get_ticks() - start_time) // 1000
    remaining_time = max(0, time_limit - elapsed_time)

    if remaining_time == 0:
        screen.fill(BLACK)
        lose_text = font.render("Time's Up! You Lose!", True, (255, 0, 0))
        screen.blit(lose_text, (SCREEN_WIDTH // 2 - 100, SCREEN_HEIGHT // 2 - 20))
        pygame.display.flip()
        pygame.time.wait(3000)
        running = False

    # Drawing
    screen.fill(BLACK)
    draw_maze(maze)
    screen.blit(player_img, (player_x * BLOCK_SIZE, player_y * BLOCK_SIZE))  # Draw player image
    screen.blit(exit_img, (exit_x * BLOCK_SIZE, exit_y * BLOCK_SIZE))  # Draw exit image

    # Display Timer
    timer_text = font.render(f"Time: {remaining_time}", True, WHITE)
    screen.blit(timer_text, (10, 10))

    pygame.display.flip()
    pygame.time.Clock().tick(30)

# Quit Pygame
pygame.quit()
sys.exit()
